In [1]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn.externals.six import StringIO



/home/sina/CMPT459/kaggle-data-analysis/env/lib/python3.8/site-packages/sklearn/externals/six.py:28: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  warnings.warn("The module is deprecated in version 0.21 and will be removed "


In [2]:
features = [
    'bathrooms', 
    'bedrooms', 
    'latitude', 
    'longitude', 
    'price',
    'Special Characters',
    'Numbers', 
    'StopWord',
    'Uppercase',
]


In [3]:
df = pd.read_json('cleaned_train.json')



df_X = df.drop('interest_level', axis=1)
df_y = df['interest_level']

df_X = df.drop('interest_level', axis=1)
df_X = df_X.drop('description', axis=1)
df_X = df_X.drop('display_address', axis=1)
df_X = df_X.drop('street_address', axis=1)
df_X = df_X.drop('features', axis=1)
df_X = df_X.drop('created', axis=1)
df_X = df_X.drop('listing_id', axis=1)


In [4]:
from sklearn.model_selection import KFold # import KFold

kf = KFold(n_splits=5) # Define the split - into 5 folds 
kf.get_n_splits(df_X) # returns the number of splitting iterations in the cross-validator

KFold(n_splits=5, random_state=None, shuffle=False)

KFold(n_splits=5, random_state=None, shuffle=False)

In [5]:
df = pd.read_json('cleaned_train.json')
df.head()


,bathrooms,bedrooms,created,description,display_address,features,latitude,listing_id,longitude,price,street_address,interest_level,Special Characters,Numbers,StopWord,Uppercase
4,1.0,1,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,2400,145 Borinquen Place,medium,20,2,8,45
6,1.0,2,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,3800,230 East 44th,low,9,2,40,71
9,1.0,2,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,3495,405 East 56th Street,medium,11,2,28,62
10,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,3000,792 Metropolitan Avenue,medium,8,1,8,61
15,1.0,0,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,2795,340 East 34th Street,low,6,1,0,36


In [13]:
df_y2 = df_y

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y2, test_size=0.2, random_state=0)
#Maybe remove test_size and set random states to 1

In [15]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)



In [16]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7142420226322764


In [18]:
###Classifier

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



from sklearn.ensemble import RandomForestClassifier

RFM = RandomForestClassifier(n_estimators=20, random_state=0)
RFM.fit(X_train, y_train)
y_pred = RFM.predict(X_test)

In [22]:
#Evaluation

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz 

print("Score: ", metrics.accuracy_score(y_test, y_pred))

#print(confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred))
#print(accuracy_score(y_test, y_pred))

Score:  0.7106738709348558


In [30]:
####more scores
scores = cross_val_score(estimator=RFM, X=df_X, y=df_y, cv=10, n_jobs=4, scoring="f1_weighted")
scores


array([0.68031763, 0.70808139, 0.68554066, 0.68845788, 0.68385995,
       0.67925306, 0.68351051, 0.67256167, 0.6801977 , 0.67642481])

In [31]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='weighted')

0.6809773110460164

In [29]:
######### normalizing

from sklearn import preprocessing

x = df_X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_X_normalized = pd.DataFrame(x_scaled)





#df_X_normalized

X_train2, X_test2, y_train2, y_test2 = train_test_split(df_X_normalized, df_y2, random_state=1)


sc_normal = StandardScaler()
X_train2 = sc_normal.fit_transform(X_train2)
X_test2 = sc_normal.transform(X_test2)


RFM_normal = RandomForestRegressor(n_estimators=20, random_state=0)
RFM_normal.fit(X_train2, y_train2)
y_pred2 = RFM_normal.predict(X_test2)

In [30]:
#Normalized score
print(confusion_matrix(y_test2, y_pred2))
print(classification_report(y_test2, y_pred2))
print(accuracy_score(y_test2, y_pred2))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [32]:
def create_kaggle_response(classifier, csv_name):
    grid_ans_cv = classifier.predict_proba(test_df)
    p1 = [ans[0] for ans in grid_ans_cv]
    p2 = [ans[1] for ans in grid_ans_cv]
    p3 = [ans[2] for ans in grid_ans_cv]
    answer_svm_cv_folds = pd.DataFrame({
        "listing_id": listing_ids,
        "high": p1,
        "medium": p2,
        "low": p3
    })
    answer_svm_cv_folds.to_csv(csv_name, index=False)

In [33]:
test_df = pd.read_json('test_data_formatted.json')
test_df_unformatted = pd.read_json('test.json')
listing_ids = test_df_unformatted['listing_id']

In [34]:
create_kaggle_response(classifier=RFM, csv_name='rfm.csv')

In [35]:
from sklearn import preprocessing

x = df_X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_X_normalized = pd.DataFrame(x_scaled)





#df_X_normalized

X_train2, X_test2, y_train2, y_test2 = train_test_split(df_X_normalized, df_y, random_state=1)


sc_normal = StandardScaler()
X_train2 = sc_normal.fit_transform(X_train2)
X_test2 = sc_normal.transform(X_test2)


RFM_normal = RandomForestClassifier(n_estimators=20, random_state=0)
RFM_normal.fit(X_train2, y_train2)
y_pred2 = RFM_normal.predict(X_test2)
#Normalized score
print("Accuracy:",metrics.accuracy_score(y_test2, y_pred2))
print(classification_report(y_test2, y_pred2))

Accuracy: 0.7160916727836228
              precision    recall  f1-score   support

        high       0.46      0.27      0.34       958
         low       0.78      0.90      0.84      8568
      medium       0.44      0.29      0.35      2735

    accuracy                           0.72     12261
   macro avg       0.56      0.49      0.51     12261
weighted avg       0.68      0.72      0.69     12261



In [36]:
create_kaggle_response(classifier=RFM_normal, csv_name='rfm_normal.csv')